# California House Price Prediction - Quy trình CRISP-DM

## 1. Business Understanding
**Mục tiêu:** Dự đoán giá nhà trung bình (Median House Value) tại California dựa trên các đặc điểm như vị trí, số phòng, dân số, thu nhập, v.v.

**KPI:** R2 Score, RMSE (Root Mean Squared Error), MAE (Mean Absolute Error).

**Giá trị mang lại:** Hỗ trợ các công ty bất động sản và nhà đầu tư định giá bất động sản chính xác hơn, từ đó đưa ra quyết định đầu tư hiệu quả.

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Thêm thư mục src vào đường dẫn hệ thống để import các module
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from preprocessing import load_data, preprocess_data
from modeling import train_model, evaluate_model, save_model
from predict import predict_new_data

## 2. Data Understanding
Tải dữ liệu và khám phá sơ bộ cấu trúc, kiểu dữ liệu và phân phối.

In [ ]:
# Tải dữ liệu
DATA_PATH = '../data/housing.csv'
df = load_data(DATA_PATH)
df.head()

In [ ]:
df.info()

### EDA Highlights
- Kiểm tra phân phối của biến mục tiêu `median_house_value`.
- Xem xét mối quan hệ giữa các biến số (ví dụ: `median_income`) và giá nhà.

In [ ]:
# Trực quan hóa phân phối của biến mục tiêu
plt.figure(figsize=(10, 6))
sns.histplot(df['median_house_value'], bins=50, kde=True)
plt.title('Phân phối của Giá nhà trung bình (Median House Value)')
plt.xlabel('Giá trị')
plt.ylabel('Tần suất')
plt.show()

## 3. Data Preparation
Sử dụng hàm `preprocess_data` từ module `src` để thực hiện các bước:
- Xử lý giá trị thiếu (Imputation).
- Mã hóa biến phân loại (One-Hot Encoding).
- Chuẩn hóa dữ liệu số (StandardScaler).
- Chia tập dữ liệu thành Train/Test.

In [ ]:
X_train_processed, X_test_processed, y_train, y_test, preprocessor, feature_names = preprocess_data(df, target_col='median_house_value')

print("Kích thước tập huấn luyện:", X_train_processed.shape)
print("Kích thước tập kiểm tra:", X_test_processed.shape)

## 4. Modeling
Sử dụng **GridSearchCV** (được tích hợp trong hàm `train_model`) để tìm tham số tốt nhất cho các mô hình:
1. Linear Regression
2. Random Forest

In [ ]:
# Huấn luyện mô hình Linear Regression
lr_model, lr_params = train_model(X_train_processed, y_train, model_type='linear_regression')
print("Tham số tốt nhất cho Linear Regression:", lr_params)

In [ ]:
# Huấn luyện mô hình Random Forest
rf_model, rf_params = train_model(X_train_processed, y_train, model_type='random_forest')
print("Tham số tốt nhất cho Random Forest:", rf_params)

## 5. Evaluation
Đánh giá hiệu suất của các mô hình trên tập kiểm tra bằng các chỉ số R2, RMSE và MAE.

In [ ]:
print("--- Đánh giá Linear Regression ---")
lr_metrics = evaluate_model(lr_model, X_test_processed, y_test)
print(lr_metrics)

print("\n--- Đánh giá Random Forest ---")
rf_metrics = evaluate_model(rf_model, X_test_processed, y_test)
print(rf_metrics)

### Error Analysis
Phân tích lỗi dự đoán bằng cách so sánh giá trị thực tế và giá trị dự đoán.

In [ ]:
# Trực quan hóa Giá trị thực tế vs Dự đoán cho Random Forest
y_pred_rf = rf_model.predict(X_test_processed)

plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred_rf, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Giá trị thực tế')
plt.ylabel('Giá trị dự đoán')
plt.title('Random Forest: Thực tế vs Dự đoán')
plt.show()

## 6. Deployment
Lưu mô hình tốt nhất và bộ tiền xử lý (preprocessor) để sử dụng cho ứng dụng demo hoặc triển khai thực tế.

In [ ]:
import joblib

# Đảm bảo thư mục models tồn tại
os.makedirs('../models', exist_ok=True)

# Lưu mô hình Random Forest (mô hình tốt hơn)
save_model(rf_model, '../models/rf_model.pkl')
joblib.dump(preprocessor, '../models/preprocessor.pkl')

print("Mô hình và bộ tiền xử lý đã được lưu vào thư mục ../models/")